In [1]:
import pygame
import numpy as np
from scipy import linalg
from sklearn.neural_network import MLPRegressor
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

pygame 2.1.2 (SDL 2.0.18, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class InvertedPendulum():
    # Initialize environment.
    def __init__(self, xRef = 0.0, randomParameters = False, randomSensor = False, randomActuator = False):
        # System parameters.
        self.tau = 0.01
        if not randomParameters:
            self.g = 9.8
            self.M = 1.0
            self.m = 0.1
            self.l = 0.5
        else:
            self.g = 9.8 + 0.098*np.random.randn()
            self.M = 1.0 + 0.1 *np.random.randn()
            self.m = 0.1 + 0.01*np.random.randn()
            self.l = 0.5 + 0.05*np.random.randn()
            
        self.xRef = xRef

        # Drawing parameters.
        self.cartWidth = 80
        self.cartHeight = 40
        self.pendulumLength = 200
        self.baseLine = 350
        self.screenWidth = 800
        self.screenHeight = 400
        
        # Variable to see if simulation ended.
        self.finish = False
        
        # Variable to see if there is randomness in the sensors and actuators.
        self.randomSensor   = randomSensor
        self.randomActuator = randomActuator
        
        # Create a random observation.
        self.reset()

        # Create screen.
        self.screen = pygame.display.set_mode((self.screenWidth, self.screenHeight))
        pygame.display.set_caption('Inverted Pendulum')
        self.screen.fill('White')
        
        # Create a clock object.
        self.clock = pygame.time.Clock()
        pygame.display.update()

    # Close environment window.
    def close(self):
        pygame.quit()
        
    # Reset system with a new random initial position.
    def reset(self):
        self.observation = np.random.uniform(low = -0.05, high = 0.05, size = (4,))
        if self.randomSensor:
            return self.noise_sensors(self.observation.copy())
        else:
            return self.observation.copy()
    
    # Insert noise on the sensors.
    def noise_sensors(self, observation, noiseVar = 0.01):
        observation[0] = observation[0] + noiseVar*np.random.randn()
        observation[1] = observation[1] + noiseVar*np.random.randn()
        observation[2] = observation[2] + noiseVar*np.random.randn()
        observation[3] = observation[3] + noiseVar*np.random.randn()
        return observation
    
    # Insert noise on actuator.
    def noise_actuator(self, action, noiseVar = 0.01):
        action += noiseVar * np.random.randn()
        return action
    
    # Display object.
    def render(self):
        # Check for all possible types of player input.
        for event in pygame.event.get():
            # Command for closing the window.
            if (event.type == pygame.QUIT):
                pygame.quit()
                self.finish = True
                return None
            
            if (event.type == pygame.KEYDOWN):
                if (event.key == pygame.K_LEFT):
                    self.xRef -= 0.01
                    
                elif (event.key == pygame.K_RIGHT):
                    self.xRef += 0.01
                    
                elif (event.key == pygame.K_SPACE):
                    self.step(200*np.random.randn())
        
        # Apply surface over display.
        self.screen.fill('White')
        pygame.draw.line(self.screen, 'Black', (0, self.baseLine), (self.screenWidth, self.baseLine))
        
        # Get position for cart.
        xCenter = self.screenHeight + self.screenHeight * self.observation[0]
        xLeft   = xCenter - self.cartWidth//2
        # xRight  = xCenter + self.cartWidth//2
        
        # Get position for pendulum.
        pendX = xCenter +  self.pendulumLength * np.sin(self.observation[2])
        pendY = self.baseLine - self.pendulumLength * np.cos(self.observation[2])
        
        # Display objects.
        pygame.draw.line(self.screen,   'Green', (int(self.screenHeight + self.xRef * self.screenHeight), 0), (int(self.screenHeight + self.xRef * self.screenHeight), self.baseLine), width = 1)
        pygame.draw.rect(self.screen,   'Black', (xLeft, self.baseLine-self.cartHeight//2, self.cartWidth, self.cartHeight),  width = 0)
        pygame.draw.line(self.screen,   (100, 10, 10),   (xCenter, self.baseLine), (pendX, pendY), width = 6)
        pygame.draw.circle(self.screen, 'Blue',  (xCenter, self.baseLine), 10)
    
        # Draw all our elements and update everything.
        pygame.display.update()
        
        # Limit framerate.
        self.clock.tick(60)

    # Perform a step.
    def step(self, force):
        if self.randomActuator:
            force = self.noise_actuator(force)
        x1 = self.observation[0]
        x2 = self.observation[1]
        x3 = self.observation[2]
        x4 = self.observation[3]
        x4dot = (self.g * np.sin(x3) - np.cos(x3) * (force + self.m * self.l * x4**2 * np.sin(x3))/(self.M + self.m)) / (self.l * (4.0/3.0 - self.m * np.cos(x3)**2 / (self.M + self.m)))
        x2dot = (force + self.m * self.l * x4**2 * np.sin(x3))/(self.M + self.m) - self.m * self.l * x4dot * np.cos(x3) / (self.M + self.m)
        self.observation[0] = x1 + self.tau * x2
        self.observation[1] = x2 + self.tau * x2dot
        self.observation[2] = x3 + self.tau * x4
        self.observation[3] = x4 + self.tau * x4dot
        if self.randomSensor:
            return self.noise_sensors(self.observation.copy())
        else:
            return self.observation.copy()

In [3]:
# Dados = pd.read_csv('G:\Outros computadores\Meu modelo Computador\IFES\9 Período\Controle Inteligente\Trabalho 3\PenduloInvertidoFuzzyArtigo.csv', on_bad_lines='skip', header=None)
Dados = pd.read_csv(r'G:\Outros computadores\Meu modelo Computador\IFES\9 Período\Controle Inteligente\Trabalho 3\Coleta_csv\teste3.csv', on_bad_lines='skip', header=None)
# Dados.values
Dados.head(5)
# print(Dados)

,0,1,2,3,4
0,-723.923833,-18910.418713,155.523129,20608.846771,-880.048028
1,-837.407040,5283.504181,1378.153157,-5890.874351,1130.027353
2,-800.373583,3865.077727,897.798890,-4328.296385,-66.624053
3,-780.124813,-3957.823702,721.207956,4230.013959,-365.793351
4,-808.241109,563.653724,710.311049,-720.552470,213.273400


In [4]:
from sklearn.utils import column_or_1d


Entradas = Dados.iloc[:,:-1]
Saidas = Dados.iloc[:,4]
# Saidas = column_or_1d(Saidas, warn=True)
# Entradas.head()
# Saidas.head()


In [5]:
Rede=MLPRegressor(hidden_layer_sizes=[4,4,2,1],
                  activation='relu',
                  max_iter=10000,
                  solver="adam")
Rede.fit(Entradas,Saidas)

MLPRegressor(hidden_layer_sizes=[4, 4, 2, 1], max_iter=10000)

In [6]:
# # Modelo.save('G:\Outros computadores\Meu modelo Computador\IFES\9 Período\Controle Inteligente\Trabalho 3\ RedeTreinada.h5')

filename = 'Rede.sav'
pickle.dump(Rede, open(filename, 'wb'))

In [7]:
Rede = pickle.load(open('G:\Outros computadores\Meu modelo Computador\IFES\9 Período\Controle Inteligente\Trabalho 3\Rede.sav', 'rb'))

In [25]:
g = 9.8
M = 1.0
m = 0.1
l = 0.5
L = 2*l
I = m*L**2 / 12

# SENSORES.
# sensores[0]: posição.
# sensores[1]: velocidade.
# sensores[2]: ângulo.
# sensores[3]: velocidade angular.
# SETPOINT em env.xRef.
k1=-2416
k2=1087
k3=4789
k4=1482

Sensores_sim=np.zeros(4)
# Função de controle: Ação nula.
def funcao_controle_1(sensores):
    Sensores_sim[0]=(env.xRef-sensores[0])*k1
    Sensores_sim[1]=sensores[1]*k2
    Sensores_sim[2]=sensores[2]*k3
    Sensores_sim[3]=sensores[3]*k4
    acao=Rede.predict([Sensores_sim])
    print(acao)

    # --------------------------------------------------------------------

    # loaded_model = pickle.load(open(filename, 'rb'))
    # result = loaded_model.score(sensores[0], sensores[1], sensores[2], sensores[3])
    # acao = result

    # --------------------------------------------------------------------

    return acao


# Cria o ambiente de simulação.
env = InvertedPendulum(0.50)

# Reseta o ambiente de simulação.
sensores = env.reset()

while True:
    # Renderiza o simulador.
    env.render()
    if env.finish:
        break
    
    # Calcula a ação de controle.
    acao = funcao_controle_1(sensores)  # É ESSA A FUNÇÃO QUE VOCÊS DEVEM PROJETAR.
    
    # Aplica a ação de controle.
    sensores = env.step(acao)
    
env.close()

[-170.92694304]
[-39.03044544]
[1.63021743]
[70.98428899]
[1.63021743]
[1.63021743]
[6.73073165]
[54.47773383]
[1.63021743]
[1.63021743]
[1.63021743]
[44.99368443]
[1.63021743]
[1.63021743]
[1.63021743]
[16.3931756]
[1.63021743]
[21.17032156]
[1.63021743]
[1.63021743]
[18.82645997]
[1.63021743]
[1.63021743]
[8.80457047]
[1.63021743]
[13.93482362]
[1.63021743]
[1.63021743]
[3.06379651]
[8.94000136]
[1.63021743]
[1.63021743]
[7.39086936]
[1.63021743]
[1.63021743]
[60.89777666]
[1.63021743]
[1.63021743]
[1.63021743]
[1.63021743]
[1.63021743]
[1.63021743]
[1.63021743]
[-17.74098097]
[1.63021743]
[-8.65349181]
[-1.06208888]
[-9.43483022]
[1.63021743]
[-10.34007362]
[1.63021743]
[-5.69826147]
[0.32789321]
[-6.7866074]
[1.63021743]
[-6.41763879]
[1.63021743]
[-5.5307225]
[1.63021743]
[-5.75912485]
[1.63021743]
[-4.65353798]
[1.63021743]
[-5.60965993]
[1.63021743]
[-3.62463083]
[1.29145143]
[-5.81391953]
[12.43064945]
[-24.64074798]
[32.97502595]
[-35.48116847]
[26.97177609]
[-24.53554465]
[10